In [ ]:
!pip install transformers datasets --quiet 
!pip install pandas numpy nltk matplotlib gdown 

     |████████████████████████████████| 4.0 MB 7.3 MB/s 
     |████████████████████████████████| 6.6 MB 31.8 MB/s 
     |████████████████████████████████| 77 kB 6.8 MB/s 
     |████████████████████████████████| 880 kB 38.3 MB/s 
     |████████████████████████████████| 596 kB 29.2 MB/s 


In [ ]:
import pandas as pd 
import numpy as np 
import random
import nltk 
from nltk.tokenize import word_tokenize

import transformers 
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
seed = 42 
torch.manual_seed(seed)
np.random.seed(seed) 
random.seed(seed)

## Загружаем тексты

In [ ]:
# download prozhito texts 
import gdown

url = 'https://drive.google.com/file/d/1oHAmsa6Nxb1B2sbWriJXCExETwSirtMj/view?usp=sharing' 
gdown.download(url, 'prozhito_texts.zip', fuzzy=True)
# url = 'https://drive.google.com/file/d/1OKaDPoiS-JX1LRKYEB7o-p1v98nc1d_V/view?usp=sharing' 
# gdown.download(url, 'lenta_texts.csv', fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1oHAmsa6Nxb1B2sbWriJXCExETwSirtMj
To: /content/prozhito_texts.zip
100%|██████████| 20.8M/20.8M [00:00<00:00, 70.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1OKaDPoiS-JX1LRKYEB7o-p1v98nc1d_V
To: /content/lenta_texts.csv
100%|██████████| 41.7M/41.7M [00:00<00:00, 102MB/s]


'lenta_texts.csv'

In [ ]:
texts = [] 

### Прожито

In [ ]:
!unzip prozhito_texts.zip

In [ ]:
import glob, os

rootdir = 'prozhito_texts'
for subdir, dirs, files in os.walk(rootdir):
    for filename in files:
        if filename.endswith('.txt'): 
            path = os.path.join(subdir, filename)
            with open(path) as f: 
                sents = f.readlines() 
                sents = list(map(lambda x: x.strip(), sents))
                texts.extend([' '.join(sents)])

In [ ]:
len(texts)

16084

In [ ]:
texts[4]

'Вчера был весь вечер у Герты Михайловны Неменовой , маленькой старушки , сухонькой , плосконькой , но с лицом одухотворенным , с умными живыми глазами . Курильщица . Очень живая , говорит почти шепотом , округляя глаза . Живопись разная . Но одна — шедевр . Это странная вещь « Балерина » . Старуха в зеленой пачке , в спущенных чулках , рыжеволосая , с руками прачки , красными , натруженными . Она писала ее в конце 20-х , увезла в Париж , где жила год и три месяца на стипендию от государства . И Ларионов , и Гончарова < com id = " 1531386025145 " / > были в восторге от этой работы . Натурщицей была Полина Бернштейн < com id = " 1531386025146 " / > . — Я подошла к ней и спрашиваю : « Нет ли у Вас голубой пачки ? Она отвечает : — Зеленая есть . Знаете , я ведь училась танцевать вместе с Лилей Брик . И она надела . — Я только попросила ее : « Оставьте чулки » . Она с удовольствием позировала , но кажется я разбила ей жизнь . — « Можно придет посмотреть мой знакомый ? » — Я разрешила . И о

### Новости

In [ ]:
# df = pd.read_csv('lenta_texts.csv') 
# df.head()

,url,title,text,topic,tags,date,year
0,https://lenta.ru/news/2002/10/09/cbr/,Центробанк решил не радовать фальшивомонетчиков,Банк России не планирует в ближайший год вводи...,Экономика,Все,2002/10/09,2002
1,https://lenta.ru/news/2019/06/25/liniya/,Более 26 миллионов россиян посмотрели прямую л...,Более 26 миллионов россиян посмотрели трансляц...,NaN,Интернет,2019/06/25,2019
2,https://lenta.ru/news/2007/05/28/boxoffice/,"Третьим ""Пиратам Карибского моря"" не удалось о...","Фильму ""Пираты Карибского моря. На краю света""...",Культура,Все,2007/05/28,2007
3,https://lenta.ru/news/2009/03/13/broken/,Половину проверенных МиГ-29 признали непригодн...,Около 90 истребителей МиГ-29 забракованы в ход...,Наука и техника,Все,2009/03/13,2009
4,https://lenta.ru/news/2009/05/04/third/,По делу о бойне в Баку задержан второй человек,Азербайджанские правоохранительные органы заде...,Бывший СССР,Все,2009/05/04,2009


In [ ]:
# texts.extend(df['text'].values)

In [ ]:
# texts[-1]

'Счетная палата начала проверку Банка Москвы, входящего в пятерку крупнейших банков страны. Об этом сообщает РИА Новости со ссылкой на аудитора Счетной палаты Михаила Бесхмельницына. "Буквально в пятницу принято решение и уже разработана программа по проверке Банка Москвы, причем мы думаем провести комплексную проверку - его деятельности как банка, так и его участия в реализации программ по городу Москве, в том числе как агента", - сказал Бесхмельницын. С чем связана проверка Банка Москвы, аудитор не уточнил. Банк Москвы был создан в марте 1995 года правительством столицы. Сейчас столице напрямую принадлежит 46,48 процента акций. Еще 17,32 процента финансовой организации чиновники контролируют через группу дочерних компаний ОАО "Столичная страховая группа". Президент банка Андрей Бородин совместно с Львом Алалуевым владеют долей в 20,32 процента. Ранее российские деловые СМИ сообщали, что новое руководство Москвы начало переговоры по продаже части акций "Банка Москвы" государственной ф

In [ ]:
# len(texts)

32104

### Перемешиваем тексты

In [ ]:
random.shuffle(texts) 
texts[-1]

'Правительство России определило размер призовых для российских спортсменов, которые станут чемпионами и призерами зимней Олимпиады 2010 года. За золотую медаль олимпийцы получат 100 тысяч евро, за серебряную - 60 тысяч, за бронзовую - 40 тысяч, пишет издание "Советский спорт". Такие же призовые были на Играх-2008 в Пекине. По сравнению с предыдущей зимней Олимпиадой, которая прошла в 2006 году в Турине, призовые российским спортсменам увеличены почти в три раза. О размере призовых было объявлено 16 декабря на заседании президиума коллегии Министерства спорта, туризма и молодежной политики России. Зимняя Олимпиада 2010 года пройдет 12-28 февраля в Ванкувере. На Играх в Турине сборная России завоевала восемь золотых, шесть серебряных и восемь бронзовых медалей, заняв четвертое место в неофициальном командном зачете.'

## Загружаем машины

In [ ]:
!pip install datasets --quiet 

     |████████████████████████████████| 325 kB 5.0 MB/s 
     |████████████████████████████████| 136 kB 40.6 MB/s 
     |████████████████████████████████| 212 kB 38.7 MB/s 
     |████████████████████████████████| 1.1 MB 39.8 MB/s 
     |████████████████████████████████| 127 kB 37.9 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 271 kB 54.1 MB/s 
     |████████████████████████████████| 144 kB 52.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
from datasets import Dataset, DatasetDict, load_metric

In [ ]:
MODEL_PATH = "DeepPavlov/rubert-base-cased"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH) 
model = AutoModelForMaskedLM.from_pretrained(MODEL_PATH) 

Downloading:   0%|          | 0.00/341 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/632 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/235k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/457k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/45.5M [00:00<?, ?B/s]

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
df = pd.DataFrame({
    'text': texts 
})

df.head()

,text
0,"( ? ) июня . Набираю номер телефона Наты , что..."
1,Лондонская фондовая биржа (LSE) совместно с ам...
2,День рождения мамы . Подарочек ей приготовил н...
3,( ? ) января . Звонок Ерофеева . Очень хочет м...
4,"Объявленное перемирие действует , а раненых ве..."


In [ ]:
dataset = Dataset.from_pandas(df) 
dataset

Dataset({
    features: ['text'],
    num_rows: 32104
})

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

tokenized_dataset = dataset.map(tokenize_function, batched=True) 

  0%|          | 0/33 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (627 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
tokenized_dataset = tokenized_dataset.remove_columns('text')
tokenized_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 32104
})

In [ ]:
block_size = 256 

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_dataset.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=-1,
)

lm_datasets 

#0:   0%|          | 0/9 [00:00<?, ?ba/s]

#1:   0%|          | 0/9 [00:00<?, ?ba/s]

#2:   0%|          | 0/9 [00:00<?, ?ba/s]

#3:   0%|          | 0/9 [00:00<?, ?ba/s]

In [ ]:
from transformers import Trainer, TrainingArguments

model_name = MODEL_PATH.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-prozhito",
    evaluation_strategy = "no",
    learning_rate=5e-5,
    weight_decay=0.01, 
    num_train_epochs=5, 
    tokenizer=tokenizer
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
from transformers import .DataCollatorForWholeWordMask
data_collator = .DataCollatorForWholeWordMask(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets,
    data_collator=data_collator, 
    tokenizer=tokenizer
)

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 25823
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 16140


Step,Training Loss


KeyboardInterrupt: ignored